In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

code_dir = os.path.dirname(os.getcwd())
sys.path.append(code_dir)

from glob import glob
import pandas as pd
import numpy as np

import essentia.standard as es

from model.utils.audio_utils import load_audio

import IPython.display as ipd

from matplotlib import pyplot as plt

emb_dir = "../logs/emb/"

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [75]:
def analyze_result(df, idx, hop_size=0.5, segment_len=1.0):

    row = df.iloc[idx]

    seq_len = (row['seq_len']-1) * hop_size + segment_len

    gt_sequence = load_audio(row["gt_track_path"],
                            seg_start_sec=0.5*row["gt_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)

    #/home/oaraz/nextcore/fingerprinting/datasets/neural-audio-fp-dataset/music/test-query-db-500-30s/db/000
    #query_path = row["gt_track_path"].replace("/db/", "/query/")
    gt_list = row["gt_track_path"].split(os.sep)
    gt_list[-3] = "query"
    gt_list.insert(-2, "snr_00dB_10dB_1s")
    query_path = os.sep.join(gt_list)
    query_sequence = load_audio(query_path,
                            seg_start_sec=0.5*row["gt_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)
    
    pred_sequence = load_audio(row["pred_track_path"],
                            seg_start_sec=0.5*row["pred_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)

    print(f"GT: {os.path.basename(row['gt_track_path'])}")
    print(f"Seg: {row['gt_start_segment']}")
    ipd.display(ipd.Audio(gt_sequence, rate=8000, normalize=False))

    print(f"Query: {os.path.basename(query_path)}")
    print(f"Seg: {row['gt_start_segment']}")
    ipd.display(ipd.Audio(query_sequence, rate=8000, normalize=False))

    print(f"Pred: {os.path.basename(row['pred_track_path'])}")
    print(f"Seg: {row['pred_start_segment']}")
    ipd.display(ipd.Audio(pred_sequence, rate=8000, normalize=False))

    return row

## Read the Audio Files

In [3]:
dataset_dir  = "../../datasets/neural-audio-fp-dataset/"
music_dir = os.path.join(dataset_dir, "music")

test_dummy_dir = os.path.join(music_dir, "test-dummy-db-100k-full")
test_dummy_paths = glob(os.path.join(test_dummy_dir, "**/*.wav"), recursive=True)
print(len(test_dummy_paths))

test_query_dir = os.path.join(music_dir, "test-query-db-500-30s")

query_dir = os.path.join(test_query_dir, "query", "snr_00dB_10dB_1s")
query_paths = sorted(glob(os.path.join(query_dir, "**/*.wav"), recursive=True))
print(len(query_paths))

db_dir = os.path.join(test_query_dir, "db")
db_paths = sorted(glob(os.path.join(db_dir, "**/*.wav"), recursive=True))
print(len(db_paths))


100000
500
500


## Load The Analysis File of a Model

In [4]:
emb_dir = "../logs/emb_100k/"
test_ids = "equally_spaced"

#model_name = "fma-old_sched-val_shift"
#epoc = 100

model_name = "640_lamb"
epoc = 101

# model_name = "replication"
# epoc = 100

analysis_path = os.path.join(emb_dir, model_name, str(epoc), test_ids, "analysis.csv")

analysis_df = pd.read_csv(analysis_path)
print(analysis_df.shape)
analysis_df.head()

(15000, 7)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
0,0,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.490451
1,0,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.360557
2,0,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.330360
3,0,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.123164
4,0,11,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.044526


## Parse The Analysis File

In [55]:
# Correct track
correct_df = analysis_df[analysis_df['gt_track_path'] == analysis_df['pred_track_path']]
print(f"Number of sequences that are matched to the right track and right sequence: {correct_df.shape[0]} ", end="")
print(f"({100*correct_df.shape[0]/analysis_df.shape[0]:.2f}%)")
correct_df.head()

Number of sequences that are matched to the right track and right sequence: 13666 (91.11%)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
0,0,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.490451
1,0,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.360557
2,0,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.330360
3,0,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.123164
4,0,11,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,0.044526


In [56]:
# False track
false_track_df = analysis_df[analysis_df['gt_track_path'] != analysis_df['pred_track_path']]
print(f"Number of sequences that are matched to the wrong track: {false_track_df.shape[0]} ", end="")
print(f"({100*false_track_df.shape[0]/analysis_df.shape[0]:.2f}%)")
false_track_df.head()

Number of sequences that are matched to the wrong track: 1334 (8.89%)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
12,18,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,18,/home/oaraz/nextcore/fingerprinting/datasets/n...,2503,0.515698
84,154,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,36,/home/oaraz/nextcore/fingerprinting/datasets/n...,125,0.525433
90,177,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,4155,0.561842
96,186,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,122,0.577554
156,304,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,1049,0.502916


In [57]:
# Correct track, false segment
c_track_near_segment_df = analysis_df[(analysis_df['gt_track_path'] == analysis_df['pred_track_path']) & (np.abs(analysis_df['gt_start_segment'] - analysis_df['pred_start_segment']) == 1)]
print(f"Number of sequences that are matched to the correct track but with near time: {c_track_near_segment_df.shape[0]} ", end="")
print(f"({100*c_track_near_segment_df.shape[0]/analysis_df.shape[0]:.2f}%)")
c_track_near_segment_df.head()

Number of sequences that are matched to the correct track but with near time: 58 (0.39%)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
6,9,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,8,0.522764
312,608,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,18,/home/oaraz/nextcore/fingerprinting/datasets/n...,17,0.509954
888,1738,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,27,/home/oaraz/nextcore/fingerprinting/datasets/n...,28,0.496272
1074,2101,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,36,/home/oaraz/nextcore/fingerprinting/datasets/n...,35,0.568547
1177,2310,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,8,0.365706


In [58]:
c_track_f_segment_df = analysis_df[(analysis_df['gt_track_path'] == analysis_df['pred_track_path']) & (np.abs(analysis_df['gt_start_segment'] - analysis_df['pred_start_segment']) > 1)]
print(f"Number of sequences that are matched to the correct track but to a wrong time: {c_track_f_segment_df.shape[0]} ", end="")
print(f"({100*c_track_f_segment_df.shape[0]/analysis_df.shape[0]:.2f}%)")
c_track_f_segment_df.head()

Number of sequences that are matched to the correct track but to a wrong time: 99 (0.66%)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
270,531,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,8,0.588430
271,531,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,0,/home/oaraz/nextcore/fingerprinting/datasets/n...,8,0.334810
283,549,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,18,/home/oaraz/nextcore/fingerprinting/datasets/n...,50,0.378198
654,1275,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,36,/home/oaraz/nextcore/fingerprinting/datasets/n...,3,0.523427
1117,2192,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,6,0.397090


## Listen To Analysis

In [65]:
import random

919

In [84]:
row = analyze_result(false_track_df, random.randint(0, false_track_df.shape[0]))
#row = analyze_result(c_track_f_segment_df, 5)
#print(row['gt_track_path'])
#print(row['pred_track_path'])

GT: 012378.wav
Seg: 27


Query: 012378.wav
Seg: 27


Pred: 059218.wav
Seg: 578


In [69]:
row

test_id                                                           14001
seq_len                                                              11
gt_track_path         /home/oaraz/nextcore/fingerprinting/datasets/n...
gt_start_segment                                                     18
pred_track_path       /home/oaraz/nextcore/fingerprinting/datasets/n...
pred_start_segment                                                 1663
score                                                          0.193478
Name: 7126, dtype: object

In [34]:
counter = 0
for row in false_track_df.iterrows():
    if os.path.basename(row[1].gt_track_path) == os.path.basename(row[1].pred_track_path):
        counter += 1
print(counter)

2548
